In [1]:
import os
import dill
import numpy as np
from fastai.tabular.all import *
from fastai.vision.all import *
from fastai.vision.utils import get_image_files
from Ambrosia import pre_process_image
# from pathlib import Path, WindowsPath

from skimage import io
from PIL import Image

In [2]:
# wd = "/blue/hulcr/gmarais/Beetle_data/Beetle_classification_deep_data/Coccotypes_dactyliperda/Vial_16296/Subset_1"
wd = "F:\\Beetle_classification_deep_data\\Coccotypes_dactyliperda\\Vial_16296\\Subset_1"
image_name = "IMG_0052.JPG"
image_path = wd+"/"+image_name


############################

# load compound image
# image_arr = io.imread(image_path)
image_arr = np.array(Image.open(image_path))
# Segment image into smaller images
pre_process = pre_process_image(manual_thresh_buffer=0.15, image = image_arr) # use image_dir if directory of image used
pre_process.segment(cluster_num=2, 
                    image_edge_buffer=50)
# load model
# learn = load_learner(Path("/blue/hulcr/gmarais/Beetle_classifier/Models") / 'beetle_classifier.pkl', cpu=False, pickle_module=dill)
learn = load_learner("E:\\GIT_REPOS\\Beetle_classifier\\Models\\beetle_classifier.pkl", cpu=False) # use Path for Unix and WindowsPath for windows
# get class names
class_arr = np.append(np.array(learn.dls.vocab), "Unknown")
# get predictions for all segments
compound_prob_lst = []
for i in range(0,len(pre_process.col_image_lst)):
    prob_ar = np.array(learn.predict(pre_process.col_image_lst[i])[2])
    unkown_prob = unkown_prob_calc(probs=prob_ar, wedge_threshold=0.9, wedge_magnitude=1, wedge=True)
    prob_ar = np.append(prob_ar, unkown_prob)
    compound_prob_lst.append(prob_ar)
compound_prob_arr = np.array(compound_prob_lst)
compound_prob_arr

C:\Users\GCM\anaconda3\envs\BC_310\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


NotImplementedError: cannot instantiate 'PosixPath' on your system

In [ ]:
import torch

model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True).eval()

In [ ]:
import requests
from PIL import Image
from torchvision import transforms

# Download human-readable labels for ImageNet.
response = requests.get("https://git.io/JJkYN")
labels = response.text.split("\n")

def predict(inp_og):
    inp = transforms.ToTensor()(inp_og).unsqueeze(0)
    with torch.no_grad():
        prediction = torch.nn.functional.softmax(model(inp)[0], dim=0)
        confidences = {labels[i]: float(prediction[i]) for i in range(1000)}    
    return [inp_og, confidences]

In [ ]:
import gradio as gr

gr.Interface(fn=predict, 
             inputs=gr.Image(type="pil"),
             outputs=[gr.Image(type="pil"), gr.Label(num_top_classes=3)],
             examples=["lion.jpg", "cheetah.jpg"]).launch()